In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
!pip install comet_ml

In [24]:
from comet_ml import Experiment
# Создайте эксперимент с помощью вашего API ключа
experiment = Experiment(
    api_key="6Ga3e8coVWoOIpAreIY8DKEgJ",
    project_name="hotels",
    workspace="dekoz",
)

In [25]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
random_state = 42
RANDOM_SEED = random_state

In [26]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [27]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [28]:
df_train.info()

In [29]:
df_train.head(2)

In [30]:
df_test.info()

In [31]:
df_test.head(2)

In [32]:
sample_submission.head(2)

In [33]:
sample_submission.info()

In [34]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [35]:
data.info()

In [36]:
data['review_date'] = pd.to_datetime(data['review_date'], dayfirst=True)
data['year_ago'] = data['review_date'].apply(lambda x: x.year)
data['month'] = data['review_date'].apply(lambda x: x.month)
data['day'] = data['review_date'].apply(lambda x: x.day)
data.drop('review_date', axis=1, inplace=True)
data.head(3)

In [37]:
!pip install twython
!pip install nltk

In [38]:
data['lat'] = data['lat'].fillna(0, inplace=True)
data['lng'] = data['lng'].fillna(0, inplace=True)

In [39]:
data.drop('lat', axis=1, inplace=True)
data.drop('lng', axis=1, inplace=True)

In [40]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
import nltk
nltk.download([
     "names",
     "stopwords",
     "state_union",
     "twitter_samples",
     "movie_reviews",
     "averaged_perceptron_tagger",
     "vader_lexicon",
     "punkt",
 ])
senty = SentimentIntensityAnalyzer()

data['senty_neg'] = data['negative_review'].apply(lambda x: - (senty.polarity_scores(x))["compound"])
data['senty_pos'] = data['positive_review'].apply(lambda x: (senty.polarity_scores(x))["compound"])
data['senty'] = data['senty_pos'] - data['senty_neg']

In [41]:
data.nunique(dropna=False)

In [42]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

In [43]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [44]:
data.info()

In [45]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [46]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [47]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [48]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [49]:
n_estimators = 100

In [50]:
params={"random_state":random_state, 
        "n_estimators":n_estimators
 
}

In [51]:
experiment.log_parameters(params)

In [52]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=n_estimators, verbose=1, n_jobs=-1, random_state=random_state)

In [53]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [54]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
import numpy as np
 
# Функция для вычисления MAPE
def mape_func(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape
mape = mape_func(y_test, y_pred)
#mape = metrics.mean_absolute_percentage_error(y_test, y_pred)
print('MAPE:', mape)
metric = {'MAPE': mape}
# MAPE: 12.659399386313355 ntlk

In [55]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')
experiment.log_figure()

In [56]:
test_data.sample(10)

In [57]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [58]:
sample_submission

In [59]:
predict_submission = model.predict(test_data)

In [60]:
predict_submission

In [61]:
list(sample_submission)

In [62]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission_nltk.csv', index=False)
sample_submission.head(10)

In [63]:
experiment.log_metrics(metric)

In [64]:
experiment.end()